- DB서버의 정보
    - 주소
    - port
    - user명
    - 비밀번호
    - 데이터베이스명
- 서버의 정보들은 외부에 노출이 되면 보안적으로 문제 발생할 수 있다. 
    - 서버의 정보와 같이 외부의 노출이 되는 데이터는 dotenv를 이용하여 데이터를 숨겨서 사용 ( 환경 변수 사용 )

In [2]:
# dotenv 라이브러리 설치 
# !pip install python-dotenv

In [3]:
import pandas as pd 
import pymysql 
import os 
from dotenv import load_dotenv

In [4]:
# .env 파일을 환경 변수에 등록한다. 
load_dotenv()

True

In [6]:
# 등록된 환경 변수에 접근 
# 획득하다(get) + 환경변수(env)
# getenv( {변수명} ) -> return data의 type이 str
os.getenv('port')

'3306'

In [7]:
# DB 서버와의 연결 
# mac 사용자는 터미널에서 brew services start mysql
_db = pymysql.connect(
    host = os.getenv('host'), 
    port = int(os.getenv('port')), 
    user = os.getenv('user'), 
    password = os.getenv('pw'), 
    db = os.getenv('db_name')
) 

In [8]:
# 가상 공간 cursor 생성 ( 연결된 데이터베이스로부터 생성 )
cursor = _db.cursor( pymysql.cursors.DictCursor )

In [12]:
create_user_query = """
    CREATE TABLE if not exists `user_info`
    (
        `id` varchar(35) primary key, 
        `password` varchar(45) not null, 
        `name` varchar(45), 
        `birth` varchar(10) 
    )
"""
cursor.execute(create_user_query)

0

In [ ]:
# query문 안에 유저가 입력한 데이터를 대입하여 query 실행 
# 회원 가입을 설정 
# 입력 데이터가 4개 id, password, name, birth
input_id = input('id를 입력하시오')
input_pass = input('password를 입력하시오')
input_name = input('name을 입력하시오')
input_birth = input('birth를 입력하시오')

# 회원 정보를 저장하는 query 생성 
signup_query = """
    insert into 
    `user_info`
    values 
    ( %s, %s, %s, %s )
"""

# signup_query와 데이터를 이용하여 execute() 사용
cursor.execute( signup_query, 
               [input_id, input_pass, input_name, input_birth] )

1

- CUD(insert, update, delete)
    - query 작성
        - 데이터가 존재한다면 데이터들을 리스트나 튜플의 형태로 생성 
    - execute() 함수를 이용하여 cursor에 질의를 보낸다. 
        - cursor에 존재하는 table의 data가 변화
    - DB server와 cursor를 동기화 (.commit())
    - DB server와의 연결을 종료 
- R(select)
    - query 작성 
        - 데이터가 존재한다면 데이터들을 리스트나 튜플의 형태로 생성 
    - execute() 함수를 이용하여 cursor에 질의를 보낸다. 
        - cursor에 존재하는 table의 data의 변화 X
    - execute() 함수의 결과를 불러와서 변수에 저장 (fetchall()) 
    - DB server와의 연결을 종료

In [15]:
# query문이 select문인가?
test_qeury = """
    SELECT 
    * 
    from 
    `table`
"""

In [22]:
# query문이 select 인가?
# 좌우의 공백을 제거(strip()) -> 위치를 이용해서 문자를 나눠준다. -> 
# 문자열을 대문자, 소문자 변환 -> 비교
text_data = 'select'
test_qeury.strip().lower()[ : len(text_data) ] == text_data

True

In [26]:
# 문자를 소문자 변환(lower()) -> 
# 공백을 기준으로 데이터를 나눠준다. ( split() ) ->
# 첫번째 원소[0]를 선택 -> 비교
test_qeury.lower().split()[0] == text_data

True

In [27]:
# 문자를 소문자 변환(lower()) ->
# 좌우의 공백을 제거(strip()) ->
# 시작하는 문자가 text_data와 같은가? 함수를 호출(startswith())
test_qeury.lower().strip().startswith(text_data)

True

In [ ]:
# CRUD query를 모두 사용할 수 있는 함수를 생성 
# 매개변수 2개 -> query문, query에 대입이 되는 데이터들(가변)
def sql_qeury(
        _query, *_data_list
):
    # _query는 sql query문이 문자로 입력이되는 부분 
    # _query가 select문인가?
    if _query.lower().strip().startswith('select'):
        # cursor에 질의 보낸다. 
        cursor.execute(_query, _data_list)
        # select문에 대한 결과를 불러온다. 
        result = cursor.fetchall()
    else:
        # 질의를 보낸다. 
        cursor.execute(_query, _data_list)
        result = 'Qeury OK'

SyntaxError: expected ':' (2898232073.py, line 5)